In [1]:
import cv2
import os
import numpy as np
import imutils
import urllib.request

In [2]:
haar_file = 'haarcascade_frontalface_default.xml'
datasets = 'datasets'
face_cascade = cv2.CascadeClassifier(haar_file)
print('Training....')
(images, labels, names, id) = ([], [], {}, 0)


for (subdirs, dirs, files) in os.walk(datasets):
    for subdir in dirs:
        names[id] = subdir
        subjectpath = os.path.join(datasets, subdir)
        for filename in os.listdir(subjectpath):
            path = subjectpath + '/' + filename
            label = id
            images.append(cv2.imread(path, 0))
            labels.append(int(label))
        id += 1

(images, labels) = [np.array(lis) for lis in [images, labels]]
print(images, labels)

(width, height) = (130, 100)

model = cv2.face.LBPHFaceRecognizer_create()
# model = cv2.face.FisherFaceRecognizer_create()

# Training 
model.train(images, labels)

# Video Capture using Android mobile
url = 'http://192.168.43.1:8080/shot.jpg'

cnt = 0

while True:
    imgPath = urllib.request.urlopen(url)
    imgNp = np.array(bytearray(imgPath.read()), dtype = np.uint8)
    img = cv2.imdecode(imgNp, -1)
    img = imutils.resize(img, width = 450)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,255,0), 2)
        face = gray[y:y+h, x: x+w]
        face_resize = cv2.resize(face, (width, height))
        
        # we will provide resized images for Prediction
        # compare the Accuracy values between two images
        
        prediction = model.predict(face_resize)
        cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 3)
        if prediction[1] < 800:
            cv2.putText(img, '%s - %.0f' % (names[prediction[0]], prediction[1]), (x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0))
            print(names[prediction[0]])
            cnt = 0
        else:
            cnt += 1
            cv2.putText(img, 'Unknown', (x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0,) )
            if (cnt>100):
                print('Unknown Person')
                cv2.imwrite('unknown.jpg', img)
                cnt = 0
#     cv2.imshow('FaceRecognition', img)
    cv2.imshow('CameraFeed', img)
    key = cv2.waitKey(10)
    if key == 27:
        break
        
cv2.destroyAllWindows()

Training....


<ipython-input-2-8cf5a1b7d5d3>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  (images, labels) = [np.array(lis) for lis in [images, labels]]


[array([[ 42,  38,  34, ...,  51,  85, 133],
       [ 36,  34,  31, ...,  52,  64, 118],
       [ 39,  34,  29, ...,  52,  60, 101],
       ...,
       [ 93,  89,  85, ...,  76,  78,  83],
       [ 95,  86,  81, ...,  77,  79,  83],
       [ 95,  86,  81, ...,  75,  79,  82]], dtype=uint8)
 array([[ 24,  31,  33, ...,  42,  92, 150],
       [ 42,  39,  32, ...,  43,  63, 124],
       [ 40,  34,  31, ...,  51,  53, 108],
       ...,
       [ 91,  95,  92, ...,  77,  79,  81],
       [ 91,  96,  94, ...,  76,  78,  79],
       [ 91,  97,  96, ...,  74,  76,  78]], dtype=uint8)
 array([[ 80,  84,  84, ...,  50,  51,  51],
       [ 78,  74,  79, ...,  55,  55,  55],
       [ 76,  72,  75, ...,  59,  57,  55],
       ...,
       [160, 160, 161, ..., 149, 146, 149],
       [161, 161, 161, ..., 149, 146, 148],
       [162, 161, 161, ..., 150, 146, 148]], dtype=uint8)
 array([[ 86,  84,  81, ...,  39,  48,  54],
       [ 89,  82,  81, ...,  32,  46,  61],
       [ 87,  81,  83, ...,  25,  44, 

URLError: <urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>